# PyTorch：view() 与 reshape() 区别详解
<font size=4 color=red>
    总之，两者都是用来重塑tensor的shape的。view只适合对满足连续性条件（contiguous）的tensor进行操作，而reshape同时还可以对不满足连续性条件的tensor进行操作，具有更好的鲁棒性。view能干的reshape都能干，如果view不能干就可以用reshape来处理。别看目录挺多，但内容很细呀~其实原理并不难啦~我们开始吧~ 
</font></br>



## 主要内容

### 一、PyTorch中tensor的存储方式

    1、PyTorch张量存储的底层原理

    2、PyTorch张量的步长（stride）属性

### 二、对“视图(view)”字眼的理解

### 三、view() 和reshape() 的比较

    1、对 torch.Tensor.view() 的理解

    2、对 torch.reshape() 的理解

### 四、总结

### 一、PyTorch中tensor的存储方式
<font size=4>想要深入理解view与reshape的区别，首先要理解一些有关PyTorch张量存储的底层原理，比如tensor的头信息区（Tensor）和存储区 （Storage）以及tensor的步长Stride。不用慌，这部分的原理其实很简单的(^-^)!</font>

<font size=5>1、PyTorch张量存储的底层原理</font></br>
<font size=4>tensor数据采用头信息区（Tensor）和存储区 （Storage）分开存储的形式，如图所示。变量名以及其存储的数据是分为两个区域分别存储的。比如，我们定义并初始化一个tensor，tensor名为A，A的形状size、步长stride、数据的索引等信息都存储在头信息区（Tensor），而A所存储的真实数据则存储在存储区（Storage）。另外，如果我们对A进行截取、转置或修改等操作后赋值给B，则B的数据共享A的存储区（Storage），存储区（Storage）的数据数量没变，变化的只是B的头信息区（Tensor）对数据的索引方式。</font>
</br>
<img src="https://github.com/jhong-tao/s2dl/blob/master/ipynb/0-imgs/217856618_1_20210317091321662.jpg?raw=true" width=500 >
</br>（Tensor）


## 共用存储区域举例
<font size=4>共用存储区域，当一个变量修改存储区的数据时，另一个变量的值也会跟着变化

In [7]:
import torch
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
a = torch.arange(5)  # 初始化张量 a 为 [0, 1, 2, 3, 4]
b = a[2:]            # 截取张量a的部分值并赋值给b，b其实只是改变了a对数据的索引方式
print('a:', a)
print('b:', b)
print('id of storage of a:', id(a.storage))  # 打印a的存储区地址
print('id of storage of b:', id(b.storage))  # 打印b的存储区地址,可以发现两者是共用存储区

a: tensor([0, 1, 2, 3, 4])
b: tensor([2, 3, 4])
id of storage of a: 2175599537912
id of storage of b: 2175599537912


In [4]:
b[1] = 0    # 修改b中索引为1，即a中索引为3的数据为0
print('a:', a)
print('b:', b)
print('id of storage of a:', id(a.storage))  # 打印a的存储区地址,可以发现a的相应位置的值也跟着改变，说明两者是共用存储区
print('id of storage of b:', id(b.storage))  # 打印b的存储区地址

a: tensor([0, 1, 2, 0, 4])
b: tensor([2, 0, 4])
id of storage of a: 2175478696680
id of storage of b: 2175478696680


### PyTorch张量的步长（stride）属性
<font size=4> 
torch的tensor也是有步长（stride）属性的，说起stride属性是不是很耳熟？是的，卷积神经网络中卷积核对特征图的卷积操作也是有stride属性的，但这两个stride可完全不是一个意思哦。
</font>
<br>
<font size=4 color=red>
stride是在指定维度dim中从一个元素跳到下一个元素所必需的步长。当没有参数传入时，返回所有步长的元组。否则，将返回一个整数值作为特定维度dim中的步长。<br>
tensor的步长可以理解为从指定的维度中的一个元素到下一个元素中间的跨度。<br>为方便理解，就直接用图说明了，您细细品(^-^)：    
</font>
<img src="https://github.com/jhong-tao/s2dl/blob/master/ipynb/0-imgs/217856618_2_20210317091321818.png?raw=true"><br>
<font size=5> 
    我们来看图片上的这个例子:
</font>

In [15]:
a = torch.arange(1,10).reshape(3,-1)  # 创建一个3x3的tensor
print('a:', a)
print('stride of a:', a.stride())  # 打印a的所有维度的stride
print('stride of a0:', a.stride(0))  # 打印a的第0维度的stride，也就是表示行的维度的stride，此时行的维度索引为0，列的维度索引为1
print('stride of a1:', a.stride(1))  # 打印a的第1位的的stride，也就是表示列维度的stride

a: tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
stride of a: (3, 1)
stride of a0: 3
stride of a1: 1


<font size=4>
    从上面的结果可以看到a的第0维度的stride值为3，第1维度的stride值为1；该怎么来理解呢？a.stride(0)=3，代表的意思就是行，也就任意连续的两行的第一个值的跨度，比如第一行的第一个值1，到第二行的第一值4之间的跨度，这个跨度就是3，也就是a.stride(0)=3；同理a.stride(1)=1，代表的意思就是列，就是任意连续的两列的两个值的跨度，比如第一列的第一个值1和第二列的第二个值2之间的跨度为1，也就是a.stride(1)=1。<br>
    由此基本上说清楚了tensor的stride属性是啥意思了
</font>


## 二、对“视图(view)”字眼的理解
<font size=4>
视图是数据的一个别称或引用，通过该别称或引用亦便可访问、操作原有数据，但原有数据不会产生拷贝。如果我们对视图进行修改，它会影响到原始数据，物理内存在同一位置，这样避免了重新创建张量的高内存开销。由上面介绍的PyTorch的张量存储方式可以理解为：对张量的大部分操作就是视图操作！<br>

与之对应的概念就是副本。副本是一个数据的完整的拷贝，如果我们对副本进行修改，它不会影响到原始数据，物理内存不在同一位置。<br>

有关视图与副本，在NumPy中也有着重要的应用。可参考Numpy中的解释。
</font>
## 三、view() 和reshape() 的比较
### 1、对 torch.Tensor.view() 的理解

<font size=4>
定义：<br>
view(*shape) → Tensor<br>
作用：类似于reshape，将tensor转换为指定的形状/shape，原始的data不改变。返回的tensor与原始的tensor共享存储区。返回的tensor的size和stride必须与原始的tensor兼容。每个新的tensor的维度必须是原始维度的子空间，或满足以下连续条件：<br>
    
$$stride[i]=stride[i+1]\times size[i+1]$$

上式，为张量连续性条件，该怎么来理解这个公式呢？<br>
该公式的意思就是tensor的第i维度的stride值要等于第i+1维度的stride值乘以第i+1维度的大小<br>
否则需要先使用contiguous()方法将原始tensor转换为满足连续条件的tensor，然后就可以使用view方法进行shape变换了。或者直接使用reshape方法进行维度变换，但这种方法变换后的tensor就不是与原始tensor共享内存了，而是被重新开辟了一个空间。

如何理解tensor是否满足连续条件呐？下面通过一系列例子来慢慢理解下：

In [117]:
a = torch.arange(1, 10).reshape(3, -1)  # 3x2的tensor
a  # a的数据视图
a.storage()  # a在内存中的存放顺序
id(a.storage)  # a的数据地址
a.stride()   # a的stride
a.size()     # a的大小size

a.stride(0) == a.stride(1)*a.size(1)  # 也就是a的第0维度的stride的值是否等于a的第一维度的stride值乘以a的第一维度的大小


tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

 1
 2
 3
 4
 5
 6
 7
 8
 9
[torch.LongStorage of size 9]

2175594912008

(3, 1)

torch.Size([3, 3])

True

<font size=4>
    从上面的这个例子中我们可以看tensor a显然满足连续的条件<br>
    下面对a进行转置

In [118]:
b = a.permute(1, 0)   # 对a进行转置
b  # b的数据视图
b.storage()  # 查看不的数据存放顺序
b.storage().data_ptr()  # 查看b的数据的第一个元素的地址
a.storage().data_ptr()  # 查看b的数据的第一个元素的地址
id(b.storage) == id(a.storage)  # 判断a和b的数据存放位置是否一致


tensor([[1, 4, 7],
        [2, 5, 8],
        [3, 6, 9]])

 1
 2
 3
 4
 5
 6
 7
 8
 9
[torch.LongStorage of size 9]

2175543739136

2175543739136

True

<font size=4>
由结果可以看出，张量a、b仍然共用存储区，并且存储区数据存放的顺序没有变化，这也充分说明了b与a共用存储区，b只是改变了数据的索引方式。那么为什么b就不符合连续性条件了呐(T-T)？其实原因很简单，我们结合下图来解释：<br>
<img src="https://github.com/jhong-tao/s2dl/blob/master/ipynb/0-imgs/img23?raw=true" width=800><br>
    转置后的tensor只是对storage区数据索引方式的重映射，但原始的存放方式并没有变化.因此，这时再看tensor b的stride，从b第一行的元素1到第二行的元素2，显然在索引方式上已经不是原来+3了，而是变成了新的+1了，你在仔细琢磨琢磨是不是这样的(^-^)。所以这时候就不能用view来对b进行shape的改变了，不然就报错咯，不信你看下面;

In [119]:
c = b.reshape(9)  # reshape() 方法改变tensor形状
c
d = b.view(9)   # view()  方法改变tensor形状，报错


tensor([1, 4, 7, 2, 5, 8, 3, 6, 9])

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

<font size=4>
但是嘛，上有政策下有对策，这种情况下，直接用view不行，那我就先用contiguous()方法将原始tensor转换为满足连续条件的tensor，在使用view进行shape变换，值得注意的是，这样的原理是contiguous()方法开辟了一个新的存储区给c，并改变了c原始存储区数据的存放顺序！同样的例子：

In [128]:
# 我们来看看b的stride
b.stride()  
b.stride(0) == b.stride(1) * b.size(1)   # 显然这个时候b的stride不满足连续性条件
c = a.permute(1, 0).contiguous()  # 通过contiguous()   #方法将b的stride转为连续，此时会产生新的数据，相当于复制一份原来的数据
c.stride(0) == c.stride(1) * c.size(1)  # 此时满足了连续条件
a.storage()  # a的数据存储顺序
b.storage()  # b的数据存储顺序
c.storage()  # c的数据存储顺序
a.stride()  # stride 连续
b.stride()  # stride 不连续
c.stride()  # stride 连续，这说明contiguous方法为c另外开辟了存储区
a.storage().data_ptr()   # 查看a的数据地址起始位置，显然此时a的地址与c的地址不同
b.storage().data_ptr()   # 查看b的数据地址起始位置
c.storage().data_ptr()   # 查看c的数据地址起始位置，显然与a不同，这说明contiguous方法为c另外开辟了存储区,此时c不在与a共享存储区，修改数据互不影响


(1, 3)

False

True

 1
 2
 3
 4
 5
 6
 7
 8
 9
[torch.LongStorage of size 9]

 1
 2
 3
 4
 5
 6
 7
 8
 9
[torch.LongStorage of size 9]

...

(1, 3)

(3, 1)

2175543739136

2175543739136

2175549512576

### 2、对 torch.reshape() 的理解
<font size=4>
    定义：

torch.reshape(input, shape) → Tensor
作用：与view方法类似，将输入tensor转换为新的shape格式。

但是reshape方法更强大，可以认为a.reshape = a.view() + a.contiguous().view()。

即：在满足tensor连续性条件时，a.reshape返回的结果与a.view()相同，否则返回的结果与a.contiguous().view()相同。

不信你就看人家官方的解释嘛，您在细细品：
</font>
## 四、总结
<font size=4>
torch的view()与reshape()方法都可以用来重塑tensor的shape，区别就是使用的条件不一样。view()方法只适用于满足连续性条件的tensor，并且该操作不会开辟新的内存空间，只是产生了对原存储空间的一个新别称和引用，返回值是视图。而reshape()方法的返回值既可以是视图，也可以是副本，当满足连续性条件时返回view，否则返回副本[ 此时等价于先调用contiguous()方法在使用view() ]。因此当不确能否使用view时，可以使用reshape。如果只是想简单地重塑一个tensor的shape，那么就是用reshape，但是如果需要考虑内存的开销而且要确保重塑后的tensor与之前的tensor共享存储空间，那就使用view()。

2020.10.23

以上是我个人看了官网的的解释并实验得到的结论，所以有没有dalao知道为啥没把view废除那？是不是还有我不知道的地方

2020.11.14

为什么没把view废除那？最近偶然看到了些资料，又想起了这个问题，觉得有以下原因：

1、在PyTorch不同版本的更新过程中，view先于reshape方法出现，后来出现了鲁棒性更好的reshape方法，但view方法并没因此废除。其实不止PyTorch，其他一些框架或语言比如OpenCV也有类似的操作。

2、view的存在可以显示地表示对这个tensor的操作只能是视图操作而非拷贝操作。这对于代码的可读性以及后续可能的bug的查找比较友好。

总之，我们没必要纠结为啥a能干的b也能干，b还能做a不能干的，a存在还有啥意义的问题。就相当于马云能日赚1个亿而我不能，那我存在的意义是啥。。。存在不就是意义吗？存在即合理，最重要的是我们使用不同的方法可以不同程度上提升效率，何乐而不为？
</font>
# 参考文献
http://www.360doc.com/content/21/0317/09/7669533_967384667.shtml